# Linked Lists

In [ ]:
class ListNode:
    def __init__(self, data=0, next_node=None):
        self.data = data
        self.next = next_node
    def __repr__(self):
        return f"ListNode(data={self.data})"

def search_list(L, key):
    while L and L.data != key:
        L = L.next
    return L # returns null if key not present

def insert_after(node, new_node):
    new_node.next = node.next
    node.next = new_node

# Assuming node is not a tail
def delete_after(node):
    node.next = node.next.next

def print_list(node):
    curr = node
    while curr:
        print(curr.data, end='->')
        curr = curr.next
    print('X')

# Merge two sorted lists
Q. Consider two singly linked lists in which each node holds a number. Assume the lists are sorted. Merge the two lists so that returning list is also sorted.

In [ ]:
def merge_sorted_lists(L1, L2):
    res = ListNode()
    L3 = res
    while L1 and L2:
        if L1.data <= L2.data:
            insert_after(L3, ListNode(L1.data))
            L1 = L1.next
        else:
            insert_after(L3, ListNode(L2.data))
            L2 = L2.next
        L3 = L3.next
    while L1:
        insert_after(L3, ListNode(L1.data))
        L1 = L1.next
        L3 = L3.next
    while L2:
        insert_after(L3, ListNode(L2.data))
        L2 = L2.next
        L3 = L3.next
    return res.next

### Better Code

In [ ]:
def merge_two_sorted_lists(L1, L2):
    dummy_head = tail = ListNode()
    while L1 and L2:
        if L1.data <= L2.data:
            tail.next, L1 = L1, L1.next
        else:
            tail.next, L2 = L2, L2.next
        tail = tail.next
    tail.next = L1 or L2
    return dummy_head.next

## Reverse a single sublist
Q. Write a program which takes a single linked list L and two integers s and f as arguments, and reverses the order of the nodes from the sth node to fth node, inclusive. the numbering begins at 1 i.e., the head node is the first node. Do not allocate additional nodes.

In [ ]:
def reverse_sublist(L, s, f):
    ptr = L
    L_start = L
    count = 1
    
    while count < f:
        if count == s - 1:
            temp = ptr
            L_start = ptr.next
        ptr = ptr.next
        count += 1
    prev = ptr.next
    ptr.next = temp
    
    count = 1
    curr = L_start
    while count <= f - s + 2:
        next = curr.next
        curr.next = prev
        prev, curr = curr, next
        count += 1
L = curr = ListNode()
for i in range(10):
    insert_after(curr, ListNode(i+1))
    curr = curr.next
print_list(L)
reverse_sublist(L, 3, 9)
print_list(L)

### Better Code

In [ ]:
def reverse_sublist(L, start, finish):
    dummy_node = sublist_head = ListNode(0, L)
    for _ in range(1, start):
        sublist_head = sublist_head.next
        
    sublist_iter = sublist_head.next
    for _ in range(finish - start):
        temp = sublist_iter.next
        sublist_iter.next = temp.next
        temp.next = sublist_head.next
        sublist_head.next = temp
        
    return dummy_node.next
L = curr = ListNode()
for i in range(10):
    insert_after(curr, ListNode(i+1))
    curr = curr.next

print_list(L)
reverse_sublist(L, 3, 9)
print_list(L)

## Test for cyclicity
Q. Write a program that takes the head of the singly linked list and returns null if there does not exist a cycle, and the node at the start of the cycle, if a cycle is present.

### My try with Py
following solution uses O(n) space.

In [ ]:
def test_cyclicity(L):
    s = set()
    ptr = L
    while ptr and ptr not in s:
        s.add(ptr)
        ptr = ptr.next
    return ptr
head = curr = ListNode(1)
insert_after(curr, ListNode(2))
curr = curr.next
insert_after(curr, ListNode(3))
curr = curr.next
curr.next = head
test_cyclicity(head)

Without additional space.
keep increasing the fast iterator. If there is a cycle then fast iterator will be behind slow at one time. And that's when we return True knowing that there is a cycle. But we can't return the head of cycle.
I don't know the maths but fast iterator somehow can match or be just before the slow iterator in the loop

In [ ]:
def test_cyclicity(L):
    norm = fast = L
    count = 1
    while norm:
        speed = count
        while speed and fast:
            if fast.next == norm:
                return True
            if fast.next == None:
                return False
            fast = fast.next
            speed -= 1
        count += 1
        norm = norm.next

head = curr = ListNode(1)
insert_after(curr, ListNode(2))
curr = curr.next
insert_after(curr, ListNode(3))
curr = curr.next
insert_after(curr, ListNode(4))
curr = curr.next
curr.next = head
print(test_cyclicity(head))

### Book solution


#### BruteForce (My implementation)
we can have two iterators outer loop and inner loop.

Outer loop traverses one by one and inner loop traverses the same length traversed by outer loop. if the node is reached twice by inner loop then it contains the cycle.

Time: $O(n^2)$

In [ ]:
def has_cycle(L):
    outer = inner = L
    outer_len = 0
    while outer:
        inner_len = outer_len
        count = 0
        while inner_len:
            if inner == outer:
                count += 1
                if count == 2:
                    return inner 
            inner = inner.next
            inner_len -= 1
        # resetting inner to start of list
        inner = L
        outer = outer.next
        outer_len += 1
    return outer

head = curr = ListNode(1)
insert_after(curr, ListNode(2))
curr = curr.next
insert_after(curr, ListNode(3))
curr = curr.next
insert_after(curr, ListNode(4))
curr = curr.next
curr.next = head
print(has_cycle(head))

Now doing it in O(n) time.
We can use two iterators one fast and one slow. slow iterator steps one at a time and fast iterator steps two at a time. If there's a loop they'll meet eventually. why they'll meet? Because fast iterator will jump over the slow iterator and the slow iterator will equal the fast iterator eventually.

But the problem is finding the head of the loop. Once we know that there is a cycle we can first calculate the length of cycle say C. then we'll start from the head of list again and the first node which is equal to the node which is C after is the first element of the cycle.

In [ ]:
def has_cycle(L):
    def len_cycle(cycle_head):
        """finding the length of the cycle."""
        step = 1
        left = cycle_head
        right = left.next
        while right != left:
            step += 1
            right = right.next
        return step
    
    fast = slow = L
    while fast and fast.next and fast.next.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            cycle_len = len_cycle(slow)
            after_cycle_len = L
            for _ in range(cycle_len):
                after_cycle_len = after_cycle_len.next
            while L != after_cycle_len:
                L = L.next
                after_cycle_len = after_cycle_len.next
            return after_cycle_len
    return False
head = curr = ListNode(1)
insert_after(curr, ListNode(2))
curr = curr.next
insert_after(curr, ListNode(3))
curr = curr.next
insert_after(curr, ListNode(4))
curr = curr.next
curr.next = head
print(has_cycle(head))

# Test for overlapping lists
Q. Given two singly linked list there may be list nodes that are common to both. Write a program that takes two cycle-free singly linked lists, and determines if there exists a node that is common to both lists.

## My Try with Py
We will have two pointers for every pointer in one list another pointer will iterate through whole list and if at any point of time two of those pointer point to the same node then the linked lists are overlapping.
time- $O(n^2)$

In [ ]:
def are_overlapping(L1, L2):
    a, b = L1, L2
    while a:
        b = L2
        while b:
            if a is b:
                return True
            b = b.next
        a = a.next
    return False

head1 = curr1 = ListNode(1)
head2 = curr2 = ListNode(2)
c = ListNode(3)
d = ListNode(4)
insert_after(curr1, c)
insert_after(curr2, c)
are_overlapping(head1, head2)

Using hashtable to store nodes then another loop to see if the node is already in hashtable:

In [ ]:
def are_overlapping(l1, l2):
    used = set()
    while l1:
        used.add(l1)
        l1 = l1.next
    while l2:
        if l2 in used:
            return l2
        l2 = l2.next
    return False

head1 = curr1 = ListNode(1)
head2 = curr2 = ListNode(2)
c = ListNode(3)
d = ListNode(4)
insert_after(curr1, d)
insert_after(curr2, d)
are_overlapping(head1, head2)

If lists merge then they can't diverge so if they both have same tail then they are merged. then if we want to return the first node which is common then we can find out the length of smaller linked list and subtract it from bigger linked list's length. Subtracting them and adding one will give the position corresponding to the head of shorter one and then we can proceed further and whenever two nodes become same it is the head.

In [ ]:
def are_overlapping(l1, l2):
    if not l1 or not l2:
        return False
    len1 = len2 = 1
    cur1, cur2 = l1, l2
    while cur1.next:
        len1 += 1
        cur1 = cur1.next
    while cur2.next:
        len2 += 1
        cur2 = cur2.next
    if cur1 is cur2:
        if len1 == min(len1, len2):
            small, big = l1, l2
        else:
            small, big = l2, l1
        diff = abs(len1 - len2)
        while diff:
            big = big.next
            diff -= 1
        while big is not small:
            small = small.next
            big = big.next
        return big 
    return False
head1 = curr1 = ListNode(1)
head2 = curr2 = ListNode(2)
c = ListNode(3)
d = ListNode(4)
insert_after(curr1, c)
insert_after(curr2, d)
are_overlapping(head1, head2)

### cleaner code

In [ ]:
def overlapping_no_cycle_lists(L1, L2):
    def length(L):
        length = 0
        while L:
            length += 1
            L = L.next
        return length
    
    L1_len, L2_len = length(L1), length(L2)
    if L1_len > L2_len:
        L1, L2 = L1, L2 # L2 is the longer list
    for _ in range(abs(L1_len - L2_len)):
        L2 = L2.next
        
    while L1 and L2 and L1 is not L2:
        L1, L2 = L1.next, L2.next
    return L1
head1 = curr1 = ListNode(1)
head2 = curr2 = ListNode(2)
c = ListNode(3)
d = ListNode(4)
insert_after(curr1, d)
insert_after(curr2, d)
are_overlapping(head1, head2)

# Test for overlapping lists - lists may have cycles
Q. Question same as above but now linked list may have cycles.

#### My Try With Py

In [ ]:
def overlapping_lists_with_cycle(l1, l2):
    def has_cycle(head):
        slow = fast = head
        while fast and fast.next and fast.next.next:
            slow = slow.next
            fast = fast.next.next
            if slow is fast:
                return True, slow
        return False, None
    is_cycle_l1, l1_iter = has_cycle(l1)
    is_cycle_l2, l2_iter = has_cycle(l2)
    # if one has cycle and other doesn't
    if is_cycle_l1 ^ is_cycle_l2: 
        return False
    elif not is_cycle_l1 and not is_cycle_l2:
        head1, head2 = l1, l2
        while head1.next:
            head1 = head1.next
        while head2.next:
            head2 = head2.next
        if head1 is not head2:
            return False
        else:
            return True
    else:
        if l1_iter is l2_iter:
            return True
        head = l1_iter
        l1_iter = l1_iter.next
        while l1_iter is not head:
            if l1_iter is l2_iter:
                return True
            l1_iter = l1_iter.next
        return False
head1 = curr1 = ListNode(1)
head2 = curr2 = ListNode(2)
c = ListNode(3)
d = ListNode(4)
e = ListNode(5)
f = ListNode(6)
insert_after(curr1, c)
curr1 = curr1.next
insert_after(curr1, d)
curr1 = curr1.next
insert_after(curr1, e)
curr1 = curr1.next
insert_after(curr1, f)
curr1 = curr1.next
insert_after(curr1, d)
insert_after(curr2, d)
print(overlapping_lists_with_cycle(head1, head2))
curr1 = a = ListNode(3)
curr2 = b = ListNode(4)
c = ListNode(1)
d = ListNode(2)
insert_after(curr1, c)
curr1 = curr1.next
insert_after(curr1, a)
insert_after(curr2, d)
curr2 = curr2.next
insert_after(curr2, b)
print(overlapping_lists_with_cycle(a, b))

In [ ]:
def overlapping_lists_with_cycle(head1, head2):
    root1, root2 = has_cycle(head1), has_cycle(head2)
    if root1 ^ root2:
        return None 
    elif not root1 and not root2:
        return overlapping_no_cycle_lists(head1, head2)
    temp = root2
    while True:
        temp = temp.next
        if temp is root1 or temp is root2:
            break
    if temp is not root1:
        return None
    def distance(a, b):
        length = 0
        while a is not b:
            length += 1
            a = a.next
        return length
    head1_stem_len, head2_stem_len = distance(head1, root1), distance(head2, root2)
    #if they overlap before cycle starts
    if stem1_length > stem2_length:
        head1, head2 = head2, head1
        root1, root2 = root2, root1
    for _ in range(abs(head1_stem_len - head2_stem_len)):
        head2 = head2.next
    while head2 is not head1 and head1 is not root1 and head2 is not root2:
        head1, head2 = head1.next, head2.next
    return head1 if head1 is head2 else root1 
head1 = curr1 = ListNode(1)
head2 = curr2 = ListNode(2)
c = ListNode(3)
d = ListNode(4)
e = ListNode(5)
f = ListNode(6)
insert_after(curr1, c)
curr1 = curr1.next
insert_after(curr1, d)
curr1 = curr1.next
insert_after(curr1, e)
curr1 = curr1.next
insert_after(curr1, f)
curr1 = curr1.next
insert_after(curr1, d)
insert_after(curr2, d)
print(overlapping_lists_with_cycle(head1, head2))
curr1 = a = ListNode(3)
curr2 = b = ListNode(4)
c = ListNode(1)
d = ListNode(2)
insert_after(curr1, c)
curr1 = curr1.next
insert_after(curr1, a)
insert_after(curr2, d)
curr2 = curr2.next
insert_after(curr2, b)
print(overlapping_lists_with_cycle(a, b))

# Delete a Node

In [ ]:
def delete_from_list(head, node):
    if head is node:
        node.data = node.next.data
        node.next = node.next.next
        return
    while head and head.next is not node:
        head = head.next
    head.next = node.next
    node.next = None

def delete_from_list(node):
    if node.next is None:
        del node
        return
    node.data = node.next.data
    node.next = node.next.next

# Remove the kth last element from a list

In [ ]:
 def delete_from_last(head, k):
    l_head = head
    def length(head):
        length = 0
        while head:
            length += 1
            head = head.next
        return length
    l_len = length(head) - k
    for _ in range(0, l_len):
        head = head.next
    delete_from_list(head)
    
curr1 = a = ListNode(1)
insert_after(curr1, ListNode(2))
curr1 = curr1.next
insert_after(curr1, ListNode(3))
curr1 = curr1.next
insert_after(curr1, ListNode(4))
curr1 = curr1.next
insert_after(curr1, ListNode(5))
curr1 = curr1.next
print_list(a)
delete_from_last(a, 5)
print_list(a)

#### Amazing Approach (Mind blowwwinggggg....)
$1,2,3,4,5,6,7$

$7,6,5,4,3,2,1$

See if we want to remove the 3rd element from last. Then, we can have two iterators one pointing to the element at pos 3 and another at pos1. Now if we keep increasing both iterators maintaining the difference and when element previously at pos3 is now at end then element at pos1 will now be pointing to (k+1)th element.
We need to point to the k-th element in the end then we can just instead of pointing at pos3 we can point it at pos4 first before increasing iterators in tandem.

In [ ]:
def remove_kth_last(L, k):
    first = second = L
    for _ in range(k):
        first = first.next
    while first:
        first, second = first.next, second.next
    delete_from_list(second)
curr1 = a = ListNode(1)
insert_after(curr1, ListNode(2))
curr1 = curr1.next
insert_after(curr1, ListNode(3))
curr1 = curr1.next
insert_after(curr1, ListNode(4))
curr1 = curr1.next
insert_after(curr1, ListNode(5))
curr1 = curr1.next
print_list(a)
remove_kth_last(a, 3)
print_list(a)

# Remove Duplicates

In [ ]:
def remove_duplicates(head):
    l = r = head
    prev = 0
    while l and l.next:
        prev = l.data
        while r and r.data == prev:
            r = r.next
        l.next = r
        l = l.next
head = ListNode(2, ListNode(2, ListNode(3, ListNode(5, ListNode(7, ListNode(11, ListNode(11)))))))
head2 = ListNode(2,ListNode(2))
print_list(head2)
remove_duplicates(head2)
print_list(head2)

# Implement cyclic right shift for singly linked lists

In [ ]:
def right_shift(head, k):
    def length(head):
        count = 0
        while head:
            count += 1
            head = head.next
        return count
    k = k % length(head)
    if k == 0:
        return head
    prev, curr = None, head
    for _ in range(k):
        prev = curr
        curr = curr.next
    prev.next = None
    prev = curr
    while curr.next:
        curr = curr.next
    curr.next = head
    return prev
head = ListNode(1, ListNode(2, ListNode(3, ListNode(4, ListNode(5, ListNode(6, ListNode(7)))))))
head = ListNode(1)
print_list(head)
head = right_shift(head, 5)
print_list(head)

## Cooler Code

In [ ]:
def cyclically_right_shift_list(L, k):
    if not L:
        return L
    tail, n = L, 1
    # Calculating list length
    while tail.next:
        n += 1
        tail = tail.next
    k %= n
    if k == 0:
        return L
    
    tail.next = L
    steps_to_new_head, new_tail = n - k, tail
    while steps_to_new_head:
        steps_to_new_head -= 1
        new_tail = new_tail.next
    new_head = new_tail.next
    new_tail.next = None
    return new_head

# Even Odd Merge

In [49]:
def even_odd_merge(head):
    if not head:
        return head
    even, odd = ListNode(0), ListNode(0)
    even_ptr, odd_ptr = even, odd
    count = 0
    while head:
        if (count & 1):
            odd_ptr.next = head
            odd_ptr = odd_ptr.next
        else:
            even_ptr.next = head
            even_ptr = even_ptr.next
        count += 1
        head = head.next
    even_ptr.next = odd.next
    odd_ptr.next = None
    return even.next
head = ListNode(0, ListNode(1, ListNode(2, ListNode(3, ListNode(4, ListNode(5, ListNode(6)))))))
head = ListNode(0, ListNode(1, ListNode(2, ListNode(3, ListNode(4, ListNode(5))))))
head = ListNode(0, ListNode(1))
print_list(head)
head = even_odd_merge(head)
print_list(head)

0->X
0->X
